In [1]:
import os
from datetime import datetime, timedelta
from typing import List

from currency_converter import CurrencyConverter

from src.utils.http_request import http_request

In [2]:
fmp_api_key = os.environ.get("FMP_API_KEY")
av_api_key = os.environ.get("AV_API_KEY")

In [3]:
fmp_base_url = "https://financialmodelingprep.com/api/v3"
av_base_url = "https://www.alphavantage.co"

0. User preferences


In [4]:
stocks_of_interest = ["Apple Inc.", "Amazon.com, Inc.", "Microsoft Corporation", "Tesla, Inc."]

In [5]:
available_currencies = {
    "USD": "$",
    "EUR": "€",
}
selected_currency = "USD"
if selected_currency not in available_currencies.keys():
    raise ValueError(
        f"Currency {selected_currency} not available. Please choose from {list(available_currencies.keys())}"
    )

cur_conv = CurrencyConverter()

In [6]:
def get_stock_symbol_by_name(stock_name: str) -> str:
    fmp_api_key = os.environ.get("FMP_API_KEY")
    response = http_request(f"{fmp_base_url}/search?query={stock_name}&limit=10&apikey={fmp_api_key}").json()
    if response:
        return response[0]["symbol"]
    else:
        return None

In [7]:
symbols_of_interest = [get_stock_symbol_by_name(stock_name) for stock_name in stocks_of_interest]
symbols_of_interest

2022-10-19 14:15:58.730 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/search?query=Apple Inc.&limit=10&apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:15:59.237 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/search?query=Amazon.com, Inc.&limit=10&apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:15:59.736 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/search?query=Microsoft Corporation&limit=10&apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:00.226 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/search?query=Tesla, Inc.&limit=10&apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1


['AAPL', 'AMZN', 'MSFT', 'TSLA']

1. Stock price + change for selected stocks


In [8]:
def get_stock_price(symbol: str, currency: str = "USD") -> float:
    response = http_request(f"{fmp_base_url}/quote-short/{symbol}?apikey={fmp_api_key}").json()
    price = response[0]["price"]
    if currency != "USD":
        price = cur_conv.convert(price, "USD", currency)
    return "{:.2f}".format(price)


def get_stock_rating(symbol: str) -> str:
    response = http_request(f"{fmp_base_url}/rating/{symbol}?apikey={fmp_api_key}").json()
    rating = f"{response[0]['rating']} ({response[0]['ratingRecommendation']})"
    return rating


def get_stock_price_change(symbol: str):
    response = http_request(f"{fmp_base_url}/stock-price-change/{symbol}?apikey={fmp_api_key}").json()
    change = {
        "24h": get_percentage_change(response[0]["1D"]),
        "5D": get_percentage_change(response[0]["5D"]),
    }
    return change


def get_percentage_change(change: str) -> str:
    if float(change) > 0:
        return f"+{change}%"
    else:
        return f"{change}%"

In [9]:
for symbol in symbols_of_interest:
    change = get_stock_price_change(symbol)
    print(
        f"{symbol}: {available_currencies[selected_currency]} {get_stock_price(symbol, selected_currency)} | {change['24h']} (24h) | {change['5D']} (5 Days) | Rating: {get_stock_rating(symbol)}"
    )

2022-10-19 14:16:00.731 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/stock-price-change/AAPL?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:01.210 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/quote-short/AAPL?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:01.727 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/rating/AAPL?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1


AAPL: $ 143.75 | +0.94% (24h) | +3.32% (5 Days) | Rating: S (Strong Buy)


2022-10-19 14:16:02.375 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/stock-price-change/AMZN?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:02.843 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/quote-short/AMZN?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:03.323 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/rating/AMZN?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1


AMZN: $ 116.36 | +2.26% (24h) | +3.44% (5 Days) | Rating: A (Buy)


2022-10-19 14:16:03.791 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/stock-price-change/MSFT?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:04.407 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/quote-short/MSFT?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:04.899 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/rating/MSFT?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1


MSFT: $ 238.50 | +0.41% (24h) | +5.81% (5 Days) | Rating: S- (Strong Buy)


2022-10-19 14:16:05.395 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/stock-price-change/TSLA?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:05.881 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/quote-short/TSLA?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1
2022-10-19 14:16:06.364 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://financialmodelingprep.com/api/v3/rating/TSLA?apikey=4cea8d2e4aecdcdf7424cd6b69b37bb1


TSLA: $ 220.19 | +0.38% (24h) | +2.26% (5 Days) | Rating: A+ (Strong Buy)


2. Financial news sentiment


In [10]:
def get_news_info_by_symbol(symbol: str) -> list:
    one_day_ago = (datetime.utcnow() - timedelta(days=1)).strftime("%Y%m%dT%H%M")
    response = http_request(
        f"{av_base_url}/query?function=NEWS_SENTIMENT&tickers={symbol}&time_from={one_day_ago}&sort=latest&limit=200&apikey={av_api_key}"
    ).json()
    all_news = response["feed"]
    most_relevant_news = list(
        filter(
            lambda news: float(get_ticker_by_symbol(news["ticker_sentiment"], symbol)["relevance_score"]) >= 0.8,
            all_news,
        )
    )[:3]
    return most_relevant_news


def get_ticker_by_symbol(ticker_list, symbol):
    return next((ticker for ticker in ticker_list if ticker["ticker"] == symbol), None)


def news_analysis(symbols: List[str]):
    news_info_list = [get_news_info_by_symbol(symbol) for symbol in symbols]
    for symbol, news_info in zip(symbols, news_info_list):
        print(f"\nNews for {symbol}:")
        for news in news_info:
            symbol_ticker = get_ticker_by_symbol(news["ticker_sentiment"], symbol)
            print(
                f"The article '{news['title']}' written by {news['authors'][0]} and published on {news['source']} implies a {symbol_ticker['ticker_sentiment_label']} sentiment."
            )

In [11]:
news_analysis(symbols_of_interest)

2022-10-19 14:16:07.050 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&time_from=20221018T1216&sort=latest&limit=200&apikey=1D00HH30HPNM6NO0
2022-10-19 14:16:07.603 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AMZN&time_from=20221018T1216&sort=latest&limit=200&apikey=1D00HH30HPNM6NO0
2022-10-19 14:16:08.201 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=MSFT&time_from=20221018T1216&sort=latest&limit=200&apikey=1D00HH30HPNM6NO0
2022-10-19 14:16:08.939 | SUCCESS  | src.utils.http_request:http_request:32 - Successfully fetched data from https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=TSLA&time_from=20221018T1216&sort=latest&limit=200&apikey=1D00HH30HPNM6NO0



News for AAPL:
The article 'Apple Cuts iPhone 14 Plus Production To Tap Demand For Premium iPhone Pro Models - Apple  ( NASDAQ:AAPL ) ' written by Anusuya Lahiri and published on Benzinga implies a Somewhat-Bullish sentiment.
The article 'Apple Rounds Out Holiday Lineup With New iPads, TV Streaming Box' written by Investor's Business Daily and published on Investors Business Daily implies a Somewhat-Bullish sentiment.
The article 'Apple Unveils New Products Ahead of the Holidays. The Company Is Flexing Its Muscles, Analyst Says.' written by Angela Palumbo and published on Barrons implies a Bullish sentiment.

News for AMZN:
The article 'Amazon Gets Another Breather As Workers Dump NY Union - Amazon.com  ( NASDAQ:AMZN ) ' written by Anusuya Lahiri and published on Benzinga implies a Somewhat-Bearish sentiment.
The article 'Why I Own Amazon Stock' written by Zane Fracek and Nicholas Rossolillo and published on Motley Fool implies a Bearish sentiment.

News for MSFT:
The article 'What Ar